
# 🚀 Keystone Unsupervised Anomaly Detection — **Full Baselines + 20 Ablations (CV, Metrics, Plots, Models Saved)**

**Target runtime:** up to ~6 hours on Kaggle Free **P100 GPU**  
**What you get in one run:**
- **Baselines:** B1 Static Threshold, B2 z-Score, B3 KMeans Distance, B4 Autoencoder (PyTorch if available)
- **Core Models:** IsolationForest (IF), LOF (novelty), One-Class SVM (OCSVM), plus **Ensemble** (rank-average) and **Weighted Ensemble**
- **20 Experiments (Ablations & Analyses)** exactly as listed
- **Cross-Validation:** KFold(5) — synthetic labels generated **only on validation folds** (no leakage)
- **Metrics:** Accuracy, Precision, Recall, F1, ROC-AUC, PR-AUC, Confusion Matrix counts
- **Timing:** Fit and inference time (seconds)
- **Policy:** Allow / Step-up / Deny, full threshold grid, expected-cost curve
- **Explainability:** Permutation importance (lightweight) + optional SHAP (auto-detected)
- **Robustness:** Noise & missing value stress, adversarial-style perturbations
- **Scalability:** Subsample sizes (10k→full) reports
- **Generalization:** TimeSeriesSplit (if timestamp exists)
- **Statistical Significance:** Bootstrap test on F1
- **All models saved** under `/kaggle/working/models/<experiment_id>/`  
- **All tables** saved as CSV + one **MASTER_RESULTS_FOR_PAPER.xlsx**  
- **All plots** saved under `/kaggle/working/plots/`


## 1) Setup

In [1]:

import os, time, math, gc, warnings, json, itertools
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score,
                             confusion_matrix, roc_auc_score, average_precision_score)

from scipy.stats import rankdata

# Optional: PyTorch for Autoencoder baseline
try:
    import torch
    import torch.nn as nn
    TORCH_OK = True
except Exception:
    TORCH_OK = False

# Optional: SHAP (if present in Kaggle image). If not, we'll fall back to permutation importance.
try:
    import shap
    SHAP_OK = True
except Exception:
    SHAP_OK = False

try:
    import joblib
    JOBLIB_OK = True
except Exception:
    JOBLIB_OK = False

OUT_DIR = "/kaggle/working"
PLOT_DIR = os.path.join(OUT_DIR, "plots")
MODEL_DIR = os.path.join(OUT_DIR, "models")
os.makedirs(PLOT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

DATA_PATH = "/kaggle/input/communication-devstack-dataset/keystone_features_parsed_struct.csv"

rng = np.random.default_rng(42)

print("Torch available:", TORCH_OK, "| SHAP available:", 'Yes' if 'shap' in globals() else False, "| Joblib:", JOBLIB_OK)
print("Output dirs:", OUT_DIR, PLOT_DIR, MODEL_DIR)


Torch available: True | SHAP available: Yes | Joblib: True
Output dirs: /kaggle/working /kaggle/working/plots /kaggle/working/models


## 2) Load & Prepare (Numeric Only for Models)

In [2]:

df = pd.read_csv(DATA_PATH)
print("Loaded:", DATA_PATH, "shape:", df.shape)

# Identify timestamp column if present
TS_COL = None
for cand in ['timestamp','time','ts','datetime']:
    if cand in df.columns:
        TS_COL = cand
        break

# Numeric features for modeling
num_df = df.select_dtypes(include=[np.number]).copy()
print("Numeric features:", num_df.shape[1])

# Save EDA tables
num_df.describe().T.to_csv(os.path.join(OUT_DIR, "eda_numeric_describe.csv"))
corr = num_df.corr(numeric_only=True)
pairs = []
cols = corr.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        pairs.append((cols[i], cols[j], float(corr.iloc[i,j])))
pairs = sorted(pairs, key=lambda x: abs(x[2]), reverse=True)[:20]
pd.DataFrame(pairs, columns=['feat_a','feat_b','rho']).to_csv(os.path.join(OUT_DIR, "eda_top_corr_pairs.csv"), index=False)

# Quick hist for core columns (if present)
for c in [c for c in ['freq15_user','fail_ratio_user15','processing_time_ms'] if c in num_df.columns]:
    plt.figure(figsize=(6,4)); plt.hist(num_df[c].dropna(), bins=50)
    plt.title(f"Histogram: {c}"); plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"hist_{c}.png")); plt.close()

# Scalers
SCALERS = {'standard': StandardScaler(), 'robust': RobustScaler(), 'minmax': MinMaxScaler()}
DEFAULT_SCALER = 'standard'
X_full = SCALERS[DEFAULT_SCALER].fit_transform(num_df.values)
print("X_full:", X_full.shape, "| Scaler:", DEFAULT_SCALER)


Loaded: /kaggle/input/communication-devstack-dataset/keystone_features_parsed_struct.csv shape: (222308, 17)
Numeric features: 12
X_full: (222308, 12) | Scaler: standard


## 3) Proxy Labels, Metrics, Model I/O Helpers

In [3]:

def inject_synthetic(X, frac=0.02, scale=4.0, seed=42):
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    k = max(1, int(n*frac)); k = min(k, max(1, n//2))
    idx = rng.choice(n, size=k, replace=False)
    X_syn = X[idx].copy()
    noise = rng.normal(0.0, scale, size=X_syn.shape); X_syn += noise
    y_syn = np.ones(k, dtype=int)
    idx_norm = rng.choice(n, size=k, replace=False)
    X_norm = X[idx_norm].copy()
    y_norm = np.zeros(k, dtype=int)
    X_eval = np.vstack([X_norm, X_syn])
    y_eval = np.concatenate([y_norm, y_syn])
    return X_eval, y_eval

def norm01(a, invert=False):
    a = np.asarray(a, dtype=float)
    if invert: a = -a
    mn, mx = a.min(), a.max()
    return (a - mn) / (mx - mn + 1e-12)

def eval_scores_to_metrics(y_true, score, threshold='median'):
    s = np.asarray(score, dtype=float)
    thr = np.median(s) if threshold=='median' else float(threshold)
    y_pred = (s >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return {
        'TP': int(tp), 'TN': int(tn), 'FP': int(fp), 'FN': int(fn),
        'ACC': accuracy_score(y_true, y_pred),
        'PREC': precision_score(y_true, y_pred, zero_division=0),
        'REC': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'ROC_AUC': roc_auc_score(y_true, s),
        'PR_AUC': average_precision_score(y_true, s),
    }

def time_fit_predict(model_fit_fn, Xtr, Xte, score_fn):
    t0 = time.time(); model = model_fit_fn(Xtr); fit_t = time.time() - t0
    t1 = time.time(); score = score_fn(model, Xte); infer_t = time.time() - t1
    return model, score, fit_t, infer_t

def save_model(model, path):
    if not JOBLIB_OK: return
    os.makedirs(os.path.dirname(path), exist_ok=True)
    joblib.dump(model, path)

def save_numpy(obj, path):
    np.save(path, obj)

def save_json(obj, path):
    with open(path, "w") as f: json.dump(obj, f, indent=2)


## 4) Models & Scorers

In [4]:

from sklearn.neighbors import LocalOutlierFactor as LOFNovel

# IF
def fit_if(Xtr, contamination=0.08):
    m = IsolationForest(contamination=contamination, random_state=42)
    m.fit(Xtr); return m
def score_if(m, Xte):
    return norm01(m.decision_function(Xte), invert=True)

# LOF novelty
def fit_lof_novelty(Xtr, n_neighbors=35):
    m = LOFNovel(n_neighbors=n_neighbors, novelty=True)
    m.fit(Xtr); return m
def score_lof(m, Xte):
    return norm01(-m.score_samples(Xte), invert=False)

# OCSVM
def fit_ocsvm(Xtr, nu=0.08):
    m = OneClassSVM(nu=nu, kernel='rbf', gamma='scale')
    m.fit(Xtr); return m
def score_ocsvm(m, Xte):
    return norm01(m.decision_function(Xte), invert=True)

# KMeans baseline
def fit_kmeans(Xtr, k=8):
    m = KMeans(n_clusters=k, n_init=10, random_state=42)
    m.fit(Xtr); return m
def score_kmeans(m, Xte):
    d = m.transform(Xte).min(axis=1)
    return norm01(d, invert=False)

# z-Score baseline
def score_zscore_univariate(Xtr, Xte):
    mu = Xtr.mean(axis=0); sd = Xtr.std(axis=0) + 1e-12
    z = np.abs((Xte - mu) / sd)
    s = z.mean(axis=1)
    return norm01(s, invert=False)

# Autoencoder baseline (PyTorch)
class AE(nn.Module):
    def __init__(self, d):
        super().__init__()
        h = max(16, d//2)
        self.enc = nn.Sequential(nn.Linear(d, h), nn.ReLU(), nn.Linear(h, 8), nn.ReLU())
        self.dec = nn.Sequential(nn.Linear(8, h), nn.ReLU(), nn.Linear(h, d))
    def forward(self, x): return self.dec(self.enc(x))

def fit_autoencoder(Xtr, epochs=8, lr=1e-3, bs=256):
    if not TORCH_OK: return None
    d = Xtr.shape[1]
    model = AE(d)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    ds = torch.utils.data.TensorDataset(torch.tensor(Xtr, dtype=torch.float32))
    dl = torch.utils.data.DataLoader(ds, batch_size=bs, shuffle=True)
    model.train()
    for _ in range(epochs):
        for (xb,) in dl:
            xb = xb.to(device)
            opt.zero_grad(); loss = loss_fn(model(xb), xb); loss.backward(); opt.step()
    return model.cpu()

def score_autoencoder(model, Xte):
    if model is None: return None
    with torch.no_grad():
        x = torch.tensor(Xte, dtype=torch.float32)
        recon = model(x).numpy()
    err = ((Xte - recon)**2).mean(axis=1)
    return norm01(err, invert=False)


## 5) Cross-Validation for Baselines + Core Models (+ Save Models)

In [5]:

def run_cv_and_save(X, n_splits=5, exp_tag="core"):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rows = []
    for fold, (tr, va) in enumerate(kf.split(X), 1):
        Xtr, Xva = X[tr], X[va]
        X_eval, y_eval = inject_synthetic(Xva, frac=0.04, scale=3.5, seed=fold+999)
        base_dir = os.path.join(MODEL_DIR, f"{exp_tag}_fold{fold}")
        os.makedirs(base_dir, exist_ok=True)

        # IF
        m_if, s_if, ft_if, it_if = time_fit_predict(lambda Xt: fit_if(Xt, 0.08), Xtr, X_eval, score_if)
        save_model(m_if, os.path.join(base_dir,"IF.pkl"))
        rows.append(dict(Model='IF', Fold=fold, FitSec=ft_if, InferSec=it_if, **eval_scores_to_metrics(y_eval, s_if)))

        # LOF
        m_lof, s_lof, ft_lof, it_lof = time_fit_predict(lambda Xt: fit_lof_novelty(Xt, 35), Xtr, X_eval, score_lof)
        save_model(m_lof, os.path.join(base_dir,"LOF.pkl"))
        rows.append(dict(Model='LOF', Fold=fold, FitSec=ft_lof, InferSec=it_lof, **eval_scores_to_metrics(y_eval, s_lof)))

        # OCSVM
        m_svm, s_svm, ft_svm, it_svm = time_fit_predict(lambda Xt: fit_ocsvm(Xt, 0.08), Xtr, X_eval, score_ocsvm)
        save_model(m_svm, os.path.join(base_dir,"OCSVM.pkl"))
        rows.append(dict(Model='OCSVM', Fold=fold, FitSec=ft_svm, InferSec=it_svm, **eval_scores_to_metrics(y_eval, s_svm)))

        # KMeans
        m_km, s_km, ft_km, it_km = time_fit_predict(lambda Xt: fit_kmeans(Xt, 8), Xtr, X_eval, score_kmeans)
        save_model(m_km, os.path.join(base_dir,"KMeans.pkl"))
        rows.append(dict(Model='KMeans', Fold=fold, FitSec=ft_km, InferSec=it_km, **eval_scores_to_metrics(y_eval, s_km)))

        # z-Score (no model, save params)
        t0 = time.time(); s_z = score_zscore_univariate(Xtr, X_eval); it_z = time.time()-t0
        rows.append(dict(Model='zScore', Fold=fold, FitSec=0.0, InferSec=it_z, **eval_scores_to_metrics(y_eval, s_z)))
        if JOBLIB_OK:
            joblib.dump({'mean':Xtr.mean(axis=0), 'std':Xtr.std(axis=0)}, os.path.join(base_dir,"zscore_params.pkl"))

        # Autoencoder
        if TORCH_OK:
            m_ae, s_ae, ft_ae, it_ae = time_fit_predict(lambda Xt: fit_autoencoder(Xt, 8), Xtr, X_eval, score_autoencoder)
            if s_ae is not None:
                torch.save(m_ae.state_dict(), os.path.join(base_dir, "AE.pt"))
                rows.append(dict(Model='Autoencoder', Fold=fold, FitSec=ft_ae, InferSec=it_ae, **eval_scores_to_metrics(y_eval, s_ae)))

        # Ensemble (rank-average IF, LOF, OCSVM)
        S = np.vstack([s_if, s_lof, s_svm]).T
        ranks = np.vstack([rankdata(S[:,i]) for i in range(S.shape[1])]).T
        ens = (ranks.mean(axis=1) - ranks.min())/(ranks.max()-ranks.min()+1e-12)
        rows.append(dict(Model='Ensemble', Fold=fold, FitSec=ft_if+ft_lof+ft_svm, InferSec=it_if+it_lof+it_svm, **eval_scores_to_metrics(y_eval, ens)))

        # Weighted Ensemble (A8)
        w = np.array([0.5, 0.2, 0.3])  # IF, LOF, OCSVM
        ens_w = norm01(np.dot(S, w), invert=False)
        rows.append(dict(Model='WeightedEnsemble', Fold=fold, FitSec=ft_if+ft_lof+ft_svm, InferSec=it_if+it_lof+it_svm, **eval_scores_to_metrics(y_eval, ens_w)))

    df_cv = pd.DataFrame(rows)
    df_cv.to_csv(os.path.join(OUT_DIR, f"{exp_tag}_cv_metrics_by_fold.csv"), index=False)
    df_cv.groupby('Model').agg(['mean','std']).to_csv(os.path.join(OUT_DIR, f"{exp_tag}_cv_metrics_summary.csv"))
    return df_cv

cv_core = run_cv_and_save(X_full, 5, "core")
cv_core.head()


,Model,Fold,FitSec,InferSec,TP,TN,FP,FN,ACC,PREC,REC,F1,ROC_AUC,PR_AUC
0,IF,1,3.582123,0.104666,1568,1567,211,210,0.881609,0.881394,0.881890,0.881642,0.957722,0.954712
1,LOF,1,68.141158,1.464135,1754,1754,24,24,0.986502,0.986502,0.986502,0.986502,0.995073,0.993161
2,OCSVM,1,215.725059,2.056618,1772,1772,6,6,0.996625,0.996625,0.996625,0.996625,0.999810,0.999801
3,KMeans,1,0.962621,0.001304,1773,1773,5,5,0.997188,0.997188,0.997188,0.997188,0.998473,0.992090
4,zScore,1,0.000000,0.019973,1778,1778,0,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 6) Policy Threshold Grid & Expected Cost (A9)

In [6]:

# Fit ensemble components on FULL data (unlabeled)
iso_full = IsolationForest(contamination=0.08, random_state=42).fit(X_full)
lof_full = LocalOutlierFactor(n_neighbors=35, novelty=True).fit(X_full)
svm_full = OneClassSVM(nu=0.08, kernel='rbf', gamma='scale').fit(X_full)

def ens_scores_full():
    s_if = norm01(iso_full.decision_function(X_full), invert=True)
    s_lof = norm01(-lof_full.score_samples(X_full), invert=False)
    s_svm = norm01(svm_full.decision_function(X_full), invert=True)
    R = np.vstack([rankdata(s_if), rankdata(s_lof), rankdata(s_svm)]).T
    ens = (R.mean(axis=1) - R.min())/(R.max()-R.min()+1e-12)
    ens_w = norm01(np.dot(np.vstack([s_if,s_lof,s_svm]).T, np.array([0.5,0.2,0.3])))
    return s_if, s_lof, s_svm, ens, ens_w
s_if_full, s_lof_full, s_svm_full, ens_full, ensw_full = ens_scores_full()

def decisions_from_score(s, t1=0.30, t2=0.60):
    s = np.asarray(s)
    dec = np.full(len(s), 'Allow', dtype=object)
    dec[(s >= t1) & (s < t2)] = 'Step-up'
    dec[s >= t2] = 'Deny'
    return dec

C_false_allow, C_step_up, C_deny_false = 100.0, 2.0, 20.0
proxy_base = 0.08
thr = np.quantile(ens_full, 1 - proxy_base)
proxy_labels = (ens_full >= thr).astype(int)  # NOT used for training

def expected_cost(decisions, proxy_labels):
    normal = (proxy_labels == 0); anom = (proxy_labels == 1)
    allow = (decisions == 'Allow'); step = (decisions == 'Step-up'); deny = (decisions == 'Deny')
    return float((allow & anom).sum()*C_false_allow + step.sum()*C_step_up + (deny & normal).sum()*C_deny_false)

# Grid sweep
grid = []
for t1 in np.linspace(0.05, 0.5, 10):
    for t2 in np.linspace(t1+0.05, 0.95, 10):
        de = decisions_from_score(ens_full, t1, t2)
        d = pd.value_counts(de, normalize=True)
        grid.append((t1, t2, float(d.get('Allow',0))*100, float(d.get('Step-up',0))*100, float(d.get('Deny',0))*100, expected_cost(de, proxy_labels)))
grid_df = pd.DataFrame(grid, columns=['t1','t2','allow%','step%','deny%','expected_cost']).sort_values('expected_cost')
grid_df.to_csv(os.path.join(OUT_DIR, "policy_frontier_grid.csv"), index=False)

plt.figure(figsize=(8,4)); plt.plot(range(20), grid_df['expected_cost'].values[:20], marker='o')
plt.title("Top-20 Policies by Expected Cost"); plt.xlabel("Rank"); plt.ylabel("Expected Cost")
plt.tight_layout(); plt.savefig(os.path.join(PLOT_DIR, "policy_top20_cost.png")); plt.close()

pd.DataFrame({'Decision':['Allow','Step-up','Deny'],
              'Percent': pd.value_counts(decisions_from_score(ens_full, 0.30, 0.60), normalize=True).reindex(['Allow','Step-up','Deny']).fillna(0).values*100
             }).to_csv(os.path.join(OUT_DIR, "policy_distribution.csv"), index=False)

# Save core full-data models
if JOBLIB_OK:
    joblib.dump(iso_full, os.path.join(MODEL_DIR, "full_IF.pkl"))
    joblib.dump(lof_full, os.path.join(MODEL_DIR, "full_LOF.pkl"))
    joblib.dump(svm_full, os.path.join(MODEL_DIR, "full_OCSVM.pkl"))


## 7) Feature-Drop Ablations (A2–A5) + Scaling (A7) — CV & Save Models

In [7]:

def ablation_cv(X_df, drop_cols=None, scaler_name='standard', folds=5, exp_tag="ablation"):
    # Drop columns by name if present (from numeric df)
    if drop_cols:
        keep = [c for c in X_df.columns if c not in drop_cols]
    else:
        keep = list(X_df.columns)
    Xsub = SCALERS[scaler_name].fit_transform(X_df[keep].values)
    kf = KFold(n_splits=folds, shuffle=True, random_state=202)
    rows = []
    for fold, (tr, va) in enumerate(kf.split(Xsub), 1):
        Xtr, Xva = Xsub[tr], Xsub[va]
        X_eval, y_eval = inject_synthetic(Xva, frac=0.04, scale=3.5, seed=fold+1234)
        base_dir = os.path.join(MODEL_DIR, f"{exp_tag}_{scaler_name}_fold{fold}")
        os.makedirs(base_dir, exist_ok=True)

        # Use IF as ablation sentinel model (fast & robust)
        m_if, s_if, ft_if, it_if = time_fit_predict(lambda Xt: fit_if(Xt, 0.08), Xtr, X_eval, score_if)
        save_model(m_if, os.path.join(base_dir,"IF.pkl"))
        rows.append(dict(Experiment=exp_tag, Scaler=scaler_name, Fold=fold, Model='IF',
                         FitSec=ft_if, InferSec=it_if, **eval_scores_to_metrics(y_eval, s_if)))
    out_df = pd.DataFrame(rows)
    out_df.to_csv(os.path.join(OUT_DIR, f"{exp_tag}_{scaler_name}_cv.csv"), index=False)
    return out_df

# Define feature groups (only those existing)
BEH = [c for c in ['freq15_user','fail_ratio_user15'] if c in num_df.columns]
SYS = [c for c in ['processing_time_ms','response_size_bytes','core_switches','vars_size_bytes'] if c in num_df.columns]
SEM = [c for c in ['endpoint_class','resource_sensitivity','method'] if c in num_df.columns]
TMP = [c for c in ['hour','wday','is_off_hours'] if c in num_df.columns]

abl_tables = []
abl_tables.append(ablation_cv(num_df, drop_cols=BEH, scaler_name='standard', exp_tag="A2_drop_behavioral"))
abl_tables.append(ablation_cv(num_df, drop_cols=SYS, scaler_name='standard', exp_tag="A3_drop_system"))
abl_tables.append(ablation_cv(num_df, drop_cols=SEM, scaler_name='standard', exp_tag="A4_drop_semantic"))
abl_tables.append(ablation_cv(num_df, drop_cols=TMP, scaler_name='standard', exp_tag="A5_drop_temporal"))
for scn in ['standard','robust','minmax']:
    abl_tables.append(ablation_cv(num_df, drop_cols=None, scaler_name=scn, exp_tag=f"A7_scaler_{scn}"))

pd.concat(abl_tables, ignore_index=True).to_csv(os.path.join(OUT_DIR, "ablations_A2_A5_A7_summary.csv"), index=False)


## 8) Sensitivity (A6) — Contamination (IF) × nu (OCSVM) Grid

In [7]:

rows = []
kf = KFold(n_splits=3, shuffle=True, random_state=321)  # 3-fold to save time
for contam in [0.03,0.08,0.12]:
    for nu in [0.03,0.08,0.12]:
        for fold, (tr, va) in enumerate(kf.split(X_full), 1):
            Xtr, Xva = X_full[tr], X_full[va]
            X_eval, y_eval = inject_synthetic(Xva, frac=0.04, scale=3.5, seed=fold+500)
            # IF score
            m_if = fit_if(Xtr, contamination=contam)
            s_if = score_if(m_if, X_eval)
            # OCSVM score
            m_svm = fit_ocsvm(Xtr, nu=nu)
            s_svm = score_ocsvm(m_svm, X_eval)
            # LOF fixed
            m_lof = fit_lof_novelty(Xtr, 35); s_lof = score_lof(m_lof, X_eval)
            # Ensemble
            S = np.vstack([s_if, s_lof, s_svm]).T
            ranks = np.vstack([rankdata(S[:,i]) for i in range(S.shape[1])]).T
            ens = (ranks.mean(axis=1)-ranks.min())/(ranks.max()-ranks.min()+1e-12)
            metrics = eval_scores_to_metrics(y_eval, ens)
            rows.append(dict(contamination=contam, nu=nu, fold=fold, **metrics))
sens_df = pd.DataFrame(rows)
sens_df.to_csv(os.path.join(OUT_DIR, "A6_sensitivity_grid.csv"), index=False)
sens_pivot = sens_df.pivot_table(index='contamination', columns='nu', values='F1', aggfunc='mean')
plt.figure(figsize=(6,4)); plt.imshow(sens_pivot, aspect='auto'); plt.colorbar(label='Mean F1')
plt.xticks(range(len(sens_pivot.columns)), sens_pivot.columns); plt.yticks(range(len(sens_pivot.index)), sens_pivot.index)
plt.title("A6 Sensitivity: F1 vs (contam, nu)")
plt.tight_layout(); plt.savefig(os.path.join(PLOT_DIR, "A6_sensitivity_heatmap.png")); plt.close()


## 9) Model Disagreement (A10) — Jaccard Overlap of Top‑k%

In [9]:

def topk_set(scores, pct=1.0):
    k = max(1, int(len(scores)*pct/100.0)); return set(np.argsort(scores)[-k:])

K_list = [0.5, 1, 2, 5, 10]
rows = []
# Use full-data scores
IFs, LOFs, SVMs, ENS, ENSW = s_if_full, s_lof_full, s_svm_full, ens_full, ensw_full
for k in K_list:
    Si, Sl, Ss, Se, Sew = [topk_set(s, k) for s in [IFs, LOFs, SVMs, ENS, ENSW]]
    def j(a,b): 
        return len(a&b)/max(1,len(a|b))
    rows.append(dict(kpct=k,
                     IF_LOF=j(Si,Sl), IF_SVM=j(Si,Ss), LOF_SVM=j(Sl,Ss),
                     ENS_IF=j(Se,Si), ENS_LOF=j(Se,Sl), ENS_SVM=j(Se,Ss),
                     ENSW_ENS=j(Sew,Se)))
pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, "A10_model_disagreement.csv"), index=False)


## 10) A11 Efficiency — Already Captured in CV Fit/Infer Times

In [10]:
print('A11 captured via FitSec/InferSec in CV tables.')

A11 captured via FitSec/InferSec in CV tables.


## 11) A12/A17 Explainability — Permutation Importance (+ optional SHAP)

In [12]:

def permutation_importance(model, X, score_func, n_repeats=5, subsample=5000, seed=123):
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    idx = rng.choice(n, size=min(subsample, n), replace=False)
    Xs = X[idx].copy()
    base = score_func(model, Xs)
    base_var = np.var(base)
    importances = []
    for j in range(Xs.shape[1]):
        imp = []
        Xtmp = Xs.copy()
        for _ in range(n_repeats):
            rng.shuffle(Xtmp[:,j])
            s = score_func(model, Xtmp)
            imp.append(max(0.0, np.var(s) - base_var))
        importances.append(np.mean(imp))
    imp = np.array(importances)
    return imp / (imp.sum() + 1e-12)

# Use full IsolationForest for importance since Tree-based
imp = permutation_importance(iso_full, X_full, lambda m, Xt: norm01(m.decision_function(Xt), invert=True), n_repeats=3, subsample=4000)
feat_imp = pd.DataFrame({'feature': num_df.columns, 'perm_importance': imp}).sort_values('perm_importance', ascending=False)
feat_imp.to_csv(os.path.join(OUT_DIR, "A12_perm_importance_IF.csv"), index=False)

plt.figure(figsize=(7,6))
top = feat_imp.head(20).iloc[::-1]
plt.barh(top['feature'], top['perm_importance'])
plt.title("Permutation Importance (IF) — Top 20")
plt.tight_layout(); plt.savefig(os.path.join(PLOT_DIR, "A12_perm_importance_top20.png")); plt.close()

# Optional: SHAP for IF (TreeExplainer), if shap installed
if 'shap' in globals():
    try:
        explainer = shap.TreeExplainer(iso_full)
        samp = min(3000, X_full.shape[0])
        Xs = X_full[:samp]
        sv = explainer.shap_values(Xs)
        shap.summary_plot(sv, Xs, feature_names=list(num_df.columns), show=False)
        plt.tight_layout(); plt.savefig(os.path.join(PLOT_DIR, "A17_shap_summary_IF.png")); plt.close()
    except Exception as e:
        print("SHAP failed, used permutation importance instead:", e)


## 12) A13 Robustness — Noise / Missing Stress Tests

In [13]:

def robustness_test(X, noise_scales=[0.5,1.0,2.0], missing_fracs=[0.01,0.05,0.1]):
    rows = []
    for ns in noise_scales:
        Xn = X.copy(); Xn += rng.normal(0, ns, size=Xn.shape)
        s_if = norm01(iso_full.decision_function(Xn), invert=True)
        rows.append(dict(test='noise', level=ns, var=np.var(s_if), mean=s_if.mean()))
    for mf in missing_fracs:
        Xm = X.copy()
        m = rng.random(size=Xm.shape) < mf
        Xm[m] = np.nan
        # simple impute by column mean (avoid leakage by using full-data mean — unsupervised global)
        colmean = np.nanmean(Xm, axis=0); idxs = np.where(np.isnan(Xm))
        Xm[idxs] = np.take(colmean, idxs[1])
        s_if = norm01(iso_full.decision_function(Xm), invert=True)
        rows.append(dict(test='missing', level=mf, var=np.var(s_if), mean=s_if.mean()))
    return pd.DataFrame(rows)

rob_df = robustness_test(X_full)
rob_df.to_csv(os.path.join(OUT_DIR, "A13_robustness.csv"), index=False)


## 13) A14 Adversarial — Evasion-Style Perturbations

In [14]:

def adversarial_evasion_score(m, X, max_iter=3, eps=0.1):
    # Simple gradient-free: jitter features towards global mean to reduce anomaly score
    Xadv = X.copy()
    mu = X.mean(axis=0)
    for _ in range(max_iter):
        delta = np.sign(mu - Xadv) * eps
        Xadv = Xadv + delta
    s = norm01(m.decision_function(Xadv), invert=True)
    return s

s_base = norm01(iso_full.decision_function(X_full), invert=True)
s_adv  = adversarial_evasion_score(iso_full, X_full, max_iter=5, eps=0.02)
pd.DataFrame({'base_mean':[float(s_base.mean())], 'adv_mean':[float(s_adv.mean())]}).to_csv(os.path.join(OUT_DIR,"A14_adversarial_summary.csv"), index=False)


## 14) A15 Scalability — Subsample Sizes Timing

In [22]:

sizes = [10000, 50000, 100000, min(200000, X_full.shape[0]), X_full.shape[0]]
rows = []
for n in sizes:
    Xs = X_full[:n]
    t0=time.time(); m=IsolationForest(contamination=0.08, random_state=42).fit(Xs); t_fit=time.time()-t0
    t1=time.time(); _=m.decision_function(Xs); t_inf=time.time()-t1
    rows.append(dict(N=n, FitSec=t_fit, InferSec=t_inf))
scal_df = pd.DataFrame(rows)
scal_df.to_csv(os.path.join(OUT_DIR, "A15_scalability.csv"), index=False)
plt.figure(figsize=(7,4)); plt.plot(scal_df['N'], scal_df['FitSec'], marker='o', label='Fit')
plt.plot(scal_df['N'], scal_df['InferSec'], marker='s', label='Infer'); plt.legend()
plt.title("A15 Scalability — IF Timing vs N"); plt.xlabel("N"); plt.ylabel("Seconds")
plt.tight_layout(); plt.savefig(os.path.join(PLOT_DIR, "A15_scalability.png")); plt.close()


## 15) A16 Generalization — TimeSeriesSplit if timestamp present

In [15]:

gen_rows = []
if TS_COL is not None:
    try:
        df_sorted = df.sort_values(TS_COL).reset_index(drop=True)
        X_ts = SCALERS['standard'].fit_transform(df_sorted.select_dtypes(include=[np.number]).values)
        tscv = TimeSeriesSplit(n_splits=5)
        for fold, (tr, va) in enumerate(tscv.split(X_ts), 1):
            Xtr, Xva = X_ts[tr], X_ts[va]
            X_eval, y_eval = inject_synthetic(Xva, frac=0.04, scale=3.5, seed=fold+2121)
            m_if = fit_if(Xtr, 0.08); s_if = score_if(m_if, X_eval)
            gen_rows.append(dict(Fold=fold, **eval_scores_to_metrics(y_eval, s_if)))
        pd.DataFrame(gen_rows).to_csv(os.path.join(OUT_DIR, "A16_timeseries_generalization.csv"), index=False)
    except Exception as e:
        print("A16 TimeSeriesSplit failed:", e)
else:
    print("A16 skipped: No timestamp column detected.")


## 16) A18/A19 Metrics & Confusion Matrices — Provided by CV Tables

In [16]:
print('A18/A19 included in CV outputs core_cv_metrics_by_fold.csv & summary.')

A18/A19 included in CV outputs core_cv_metrics_by_fold.csv & summary.


## 17) A20 Statistical Significance — Bootstrap on F1

In [18]:

# Compare Ensemble vs best single (IF) using bootstrap over folds
core = pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/core_cv_metrics_by_fold.csv"))
ens = core[core['Model']=='Ensemble']['F1'].values
iff = core[core['Model']=='IF']['F1'].values
B=2000; rng = np.random.default_rng(1234); diffs=[]
for _ in range(B):
    idx = rng.integers(0, len(ens), len(ens))
    diffs.append((ens[idx] - iff[idx]).mean())
pval = (np.sum(np.array(diffs) <= 0) + 1) / (B + 1)
pd.DataFrame({'mean_diff':[float(np.mean(diffs))], 'p_value':[float(pval)]}).to_csv(os.path.join(OUT_DIR,"A20_bootstrap_ens_vs_if.csv"), index=False)


## 18) MASTER workbook export (all key outputs)

In [29]:

master_xlsx = os.path.join(OUT_DIR, "MASTER_RESULTS_FOR_PAPER.xlsx")
with pd.ExcelWriter(master_xlsx) as xl:
    # Core CV
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/core_cv_metrics_by_fold.csv")).to_excel(xl, sheet_name="core_cv_by_fold", index=False)
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/core_cv_metrics_summary.csv")).to_excel(xl, sheet_name="core_cv_summary", index=False)
    # Ablations A2-A5, A7
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/ablations_A2_A5_A7_summary.csv")).to_excel(xl, sheet_name="ablations_A2_A5_A7", index=False)
    # Policy
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/policy_frontier_grid.csv")).to_excel(xl, sheet_name="policy_frontier", index=False)
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/policy_distribution.csv")).to_excel(xl, sheet_name="policy_dist", index=False)
    # Sensitivity
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A6_sensitivity_grid.csv")).to_excel(xl, sheet_name="A6_sensitivity", index=False)
    # Disagreement
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A10_model_disagreement.csv")).to_excel(xl, sheet_name="A10_disagreement", index=False)
    # Robustness
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A13_robustness.csv")).to_excel(xl, sheet_name="A13_robustness", index=False)
    # Adversarial
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A14_adversarial_summary.csv")).to_excel(xl, sheet_name="A14_adversarial", index=False)
    # Scalability
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A15_scalability.csv")).to_excel(xl, sheet_name="A15_scalability", index=False)
    # Generalization (if exists)
    p = os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A16_timeseries_generalization.csv")
    if os.path.exists(p):
        pd.read_csv(p).to_excel(xl, sheet_name="A16_timeseries", index=False)
    # EDA
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/eda_numeric_describe.csv")).to_excel(xl, sheet_name="eda_describe", index=False)
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/eda_top_corr_pairs.csv")).to_excel(xl, sheet_name="eda_corr_pairs", index=False)
    # Importance
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A12_perm_importance_IF.csv")).to_excel(xl, sheet_name="A12_perm_importance", index=False)
    # Stats test
    pd.read_csv(os.path.join(OUT_DIR, "/kaggle/input/communication-devstack-dataset/A20_bootstrap_ens_vs_if.csv")).to_excel(xl, sheet_name="A20_bootstrap", index=False)

print("Saved MASTER workbook:", master_xlsx)


Saved MASTER workbook: /kaggle/working/MASTER_RESULTS_FOR_PAPER.xlsx


## 19) Save Full‑Data Models for Deployment

In [23]:

# Save full models used for policy/scoring
if JOBLIB_OK:
    joblib.dump(iso_full, os.path.join(MODEL_DIR, "deploy_IF.pkl"))
    joblib.dump(lof_full, os.path.join(MODEL_DIR, "deploy_LOF.pkl"))
    joblib.dump(svm_full, os.path.join(MODEL_DIR, "deploy_OCSVM.pkl"))
# Save weighted-ensemble weights
save_json({'weights': [0.5, 0.2, 0.3]}, os.path.join(MODEL_DIR, "deploy_weighted_ensemble.json"))
print("Saved deployable models in:", MODEL_DIR)


Saved deployable models in: /kaggle/working/models



## 20) Notes
- All CV metrics use **synthetic labels generated only on validation folds** to avoid leakage.  
- SHAP is optional and will be skipped if not available; **permutation importance** is always produced.  
- Expect long runtime; you asked for the **full** ablation suite with model saving.  
- All artifacts are under `/kaggle/working/`.


In [24]:
import shutil, os

# Path where Kaggle saves outputs
OUT_DIR = "/kaggle/working"

# Zip file name
zip_path = "/kaggle/working/output_results.zip"

# Remove old zip if exists
if os.path.exists(zip_path):
    os.remove(zip_path)

# Create new zip (recursively includes all files in working dir)
shutil.make_archive(zip_path.replace(".zip",""), 'zip', OUT_DIR)

print(f"✅ Zipped all outputs to {zip_path}")

✅ Zipped all outputs to /kaggle/working/output_results.zip
